In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob
import os
#import matplotlib.cm as cm
#import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
from tqdm import tqdm
import numpy as np
import functools
from copy import deepcopy
import math

from util.metrics_utils import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
res_root_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
metrics = {'f1_score': 'F1 Score'}
scenarios_dict = {'CV_20': 'computer_vision_base20_incr20_stop2',
                  '39_1': '39_1_base39_incr1_stop2'}
get_seed0 = {'39_1': True, 'CV_20': False}
max_classes = 40
selected_approaches_list = ['icarl', 'icarlp', 'bic', 'il2m', 'lwf', 'lucir', 'ewc', 'eeil', 'ssil', 
                            'chen2021', 'freezing', 'finetuning', 'backbonefreezing', 'jointft', 'joint']
selected_scenarios = ['39_1', 'CV_20']
exp_info_dict = {col: [] for col in ['Seed', 'Approach', 'Task', 'Nc first task', 'Nc incr tasks', 'Last App', 'Metric']}
per_class_met_arrival_order_dict = {col: [] for col in ['Class order[%s]' % i for i in range(max_classes)] + ['Class order']}
per_class_met_dict = {col: [] for col in ['Class %s' % i for i in range(max_classes)]}

for scenario in selected_scenarios:
    print('Scenario: %s' % scenario)
    for approach in selected_approaches_list:
        print('Approach: %s' % approach)
        exp_root_path = os.path.join(res_root_path, 'mirage_generic_%s_%s' % (approach, scenarios_dict[scenario]))
        args_filenames_list = discard_duplicates(glob(os.path.join(exp_root_path, 'args-*.txt')))
        for args_filename in tqdm(args_filenames_list):
            args_dict = get_args_dict(args_filename)
            seed = int(args_dict['seed'])
            nc_first_task = int(args_dict['nc_first_task'])
            nc_incr_tasks = int(args_dict['nc_incr_tasks'])
            last_app = str(args_dict.get('last_class', -1))
            if (seed == 0 and get_seed0[scenario]) or seed > 0:
                num_tasks = int(args_dict['stop_at_task'])
                ts = get_ts(args_filename)
                memory_size = args_dict.get('num_exemplars', 0)
                approach = args_dict['approach']\
                    if args_dict['approach'] not in ['joint', 'backbonefreezing'] or memory_size == 0\
                    else args_dict['approach'] + 'mem'
                knowledge_distillation = args_dict.get('knowledge_distillation', 'none')
                approach = approach if approach != 'lwf' or knowledge_distillation in ['tkd', 'none'] else 'lwfgkd'
                if approach != 'joint':
                    stdout_filename = os.path.join(exp_root_path, 'stdout-%s.txt' % ts)
                    stdout_values_list = get_stdout_list(stdout_filename)
                    k = 0
                    is_clorder_token = False
                    while not is_clorder_token and k < len(stdout_values_list):
                        if 'Class order:' not in stdout_values_list[k]:
                            k += 1
                        else:
                            is_clorder_token = True
                    try:
                        class_order = eval(stdout_values_list[k].split('Class order:')[1].rstrip())
                    except IndexError:
                        print('check corrupted stdout file')
                        print(stdout_filename)
                        raise
                    per_class_df = pd.read_parquet(os.path.join(exp_root_path, 'results', 'outputs_targets_features_%s-%s_per_class_metrics.parquet' % (seed, ts)))                                
                    for metric in metrics.keys():
                        for i in range(num_tasks):
                            exp_info_dict['Seed'].append(seed)
                            exp_info_dict['Approach'].append(approach)
                            exp_info_dict['Task'].append(i)
                            exp_info_dict['Nc first task'].append(nc_first_task)
                            exp_info_dict['Nc incr tasks'].append(nc_incr_tasks)
                            exp_info_dict['Last App'].append(last_app)
                            exp_info_dict['Metric'].append(metrics[metric])
                            per_class_met_arrival_order_dict['Class order'].append(class_order)
                            for cl in range(max_classes):
                                class_pos = class_order.index(cl)
                                per_class_met_arrival_order_dict['Class order[%s]' % class_pos].append(per_class_df[metric].iloc[i][class_pos] if cl in class_order[:nc_first_task+(i*nc_incr_tasks)] else np.nan)
                                per_class_met_dict['Class %s' % cl].append(per_class_df[metric].iloc[i][class_pos] if cl in class_order[:nc_first_task+(i*nc_incr_tasks)] else np.nan)

exp_info_per_cl_met_ord_arrival_dict = {**exp_info_dict, **per_class_met_arrival_order_dict}
exp_info_per_cl_met_dict = {**exp_info_dict, **per_class_met_dict}

per_cl_met_ord_arrival_df = pd.DataFrame(exp_info_per_cl_met_ord_arrival_dict)
per_cl_met_dict_df = pd.DataFrame(exp_info_per_cl_met_dict)

Scenario: 39_1
Approach: icarl


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 77.96it/s]


Approach: icarlp


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 92.59it/s]


Approach: bic


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 96.60it/s]


Approach: jointft


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 95.37it/s]


Approach: joint


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 115.10it/s]


In [4]:
#per_cl_met_ord_arrival_df.value_counts('Approach')

In [5]:
per_cl_met_ord_arrival_df

,Seed,Approach,Task,Nc first task,Nc incr tasks,Last App,Metric,Class order[0],Class order[1],Class order[2],...,Class order[31],Class order[32],Class order[33],Class order[34],Class order[35],Class order[36],Class order[37],Class order[38],Class order[39],Class order
0,0,icarl,0,39,1,14,F1 Score,0.307109,0.282071,0.605117,...,0.242424,0.039216,0.469122,0.211470,0.118110,0.269600,0.404423,0.288073,NaN,"[25, 39, 5, 28, 0, 36, 8, 20, 3, 26, 4, 9, 30,..."
1,0,icarl,1,39,1,14,F1 Score,0.260674,0.281977,0.719924,...,0.144431,0.003846,0.478859,0.253102,0.122222,0.244798,0.407643,0.291943,0.217988,"[25, 39, 5, 28, 0, 36, 8, 20, 3, 26, 4, 9, 30,..."
2,0,icarl,0,39,1,0,F1 Score,0.295455,0.424766,0.313800,...,0.494784,0.262586,0.584203,0.354884,0.344262,0.427502,0.492678,0.449136,NaN,"[10, 26, 12, 23, 21, 7, 27, 17, 35, 1, 36, 30,..."
3,0,icarl,1,39,1,0,F1 Score,0.203548,0.308511,0.251799,...,0.503794,0.280226,0.549058,0.394397,0.328725,0.400000,0.446660,0.429278,0.291585,"[10, 26, 12, 23, 21, 7, 27, 17, 35, 1, 36, 30,..."
4,0,icarl,0,39,1,22,F1 Score,0.340718,0.438972,0.070232,...,0.318928,0.264386,0.334515,0.312081,0.714575,0.448743,0.099370,0.196484,NaN,"[0, 3, 9, 17, 23, 25, 2, 16, 13, 35, 32, 37, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,33,jointmem,1,20,20,-1,F1 Score,0.373034,0.741935,0.727549,...,0.441406,0.588835,0.581024,0.382394,0.737317,0.741036,0.640809,0.734232,0.411248,"[15, 5, 0, 26, 17, 25, 36, 21, 35, 22, 34, 23,..."
2556,39,jointmem,0,20,20,-1,F1 Score,0.764552,0.902394,0.935294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[3, 2, 21, 23, 10, 16, 18, 25, 33, 39, 1, 15, ..."
2557,39,jointmem,1,20,20,-1,F1 Score,0.522659,0.569250,0.502674,...,0.380024,0.470078,0.656371,0.764392,0.934554,0.785982,0.320175,0.656347,0.348309,"[3, 2, 21, 23, 10, 16, 18, 25, 33, 39, 1, 15, ..."
2558,30,jointmem,0,20,20,-1,F1 Score,0.633555,0.936114,0.901761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[9, 21, 5, 19, 31, 8, 27, 15, 10, 29, 30, 35, ..."


In [6]:
def load_scratch_data(df_filenames, metric='f1_score', base=20, incr=5, max_classes=40):
    df_filenames = discard_duplicates(df_filenames)
    # Stop is computed basing on the base classes and the increment size up to reach the max_classes
    stop = 1 + (((max_classes - base) // incr) if incr > 0 else 0)

    df = pd.DataFrame()
    df_tmp = pd.DataFrame()
    rows = []
    for df_filename in tqdm(df_filenames):
        args_dict = get_args_dict(df_filename)
        tmp_df = pd.read_parquet(df_filename)
        
        if 'nc_first_task' in tmp_df:
            tmp_df_g = tmp_df.groupby(['nc_first_task', 'nc_incr_tasks'])
        else:
            tmp_df_g = [((args_dict['nc_first_task'], args_dict['nc_incr_tasks']), tmp_df)]

        line = None
        approach = 'scratch'
        # Retrieving the class order list from the current stdout file
        for line in get_stdout_list(df_filename):
            if 'class order' in line.lower():
                break
        class_order = eval(line.split(':')[-1])

        row = {'Seed': args_dict['seed'],
               'Last App': args_dict.get('last_class', -1),
               'Network': args_dict['network'],
               'Approach': '%s' % appr_dict[approach],
               'Batch Size': args_dict['batch_size'],
               'Patience': args_dict['lr_patience'],
               'Out Features Size': args_dict.get('out_features_size', 200),
               'Momentum': args_dict.get('momentum', .0),
               'Memory Size': args_dict.get('num_exemplars', 0),
               'App Order': class_order,
               }

        for g, tmp_d in tmp_df_g:
            _nc_first_task = g[0]
            _nc_incr_tasks = g[1]

            row.update({'Base Apps': _nc_first_task, 'Increment': _nc_incr_tasks})
            values = [[v0 for v0 in v] for v in tmp_d[metric].values]

            for ep, value in enumerate(values):
                # For each episode we store only number of apps and the list of per-class (per-episode) metrics
                row.update({
                    '#Apps': _nc_first_task + _nc_incr_tasks * ep,
                    metr_dict[metric]: value,
                })
                rows.append(deepcopy(row))
                
    # df_tmp contiene per ogni riga uno scenario (es 25+1) ed un episodio
    df_tmp = df_tmp.append(rows, ignore_index=True)
    # return df_tmp
    assert len(df_tmp) > 0
    # Basing on the passed arguments, the cumulative number of classes represents the number of classes at each episode
    cumnum_classes = list(range(base, base + incr * (stop - 1) + 1, incr)) if incr > 0 else [base]
    df_tmp = df_tmp[df_tmp['#Apps'].isin(cumnum_classes)]

    rows = []
    # This loop is needed in order to build episodes for the scratch approach
    for ep, nc in enumerate(cumnum_classes):
        # The tmp_df is grouped basing on several columns in order to separate configurations
        group_columns = ['Network', 'Out Features Size', 'Memory Size', '#Apps', 'Approach', 'Batch Size',
                         'Patience', 'Momentum']
        # The scenario 39+1 needs an alignment of per-class (per-episode) metrics
        if base == 39 and incr == 1 and stop == 2:
            # The Last App column is needed to separate different executions
            group_columns = ['Last App'] + group_columns
            df_g = df_tmp[df_tmp['#Apps'] == nc].groupby(group_columns)

            for g in df_g.groups:
                # Scratch for 39 classes and no last class is discarded, because for the base model we need the one w/
                # the correct Last App
                if g[group_columns.index('Last App')] == -1 and nc == 39:
                    continue

                values = df_tmp.loc[df_g.groups[g], 'F1 Score'].values[0]
                class_order = df_tmp.loc[df_g.groups[g], 'App Order'].values[0]
                row = dict([(k, v) for k, v in zip(group_columns, g)])
                row.update({
                    'Seed': 0,
                    'Base Apps': 39,
                    'Increment': 1,
                    'Episode': ep,
                    metr_dict[metric]: values,
                'Class order': class_order})
                rows.append(deepcopy(row))
        else:
            group_columns = ['Seed'] + group_columns
            df_g = df_tmp[df_tmp['#Apps'] == nc].groupby(group_columns)
            for g in df_g.groups:
                values = df_tmp.loc[df_g.groups[g], metr_dict[metric]].values[0]
                class_order = df_tmp.loc[df_g.groups[g], 'App Order'].values[0]
                row = dict([(k, v) for k, v in zip(group_columns, g)])
                row.update({'Last App': -1,
                            'Base Apps': base,
                            'Increment': incr,
                            'Episode': ep,
                            metr_dict[metric]: values,
                           'Class order': class_order})
                rows.append(deepcopy(row))

    df = df.append(rows, ignore_index=True)
    return df

In [14]:
scratch_df_path_dict = {(20, 20): '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/mirage_generic_scratch_*/results/outputs_targets_features_*_per_class_metrics.parquet',
                       (39, 1): '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/mirage_generic_scratch_39_ub_base39_incr1_stop1/results/outputs_targets_features_0-*_per_class_metrics.parquet',
                       (40, 0): '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/mirage_generic_scratch_*/results/outputs_targets_features_*_per_class_metrics.parquet'}
scratch_scenarios_list = [(39, 1), (40, 0), (20, 20)]
exp_info_dict = {col: [] for col in ['Seed', 'Approach', 'Task', 'Nc first task', 'Nc incr tasks', 'Last App', 'Metric']}
per_class_met_arrival_order_dict = {col: [] for col in ['Class order[%s]' % i for i in range(max_classes)] + ['Class order']}
per_class_met_dict = {col: [] for col in ['Class %s' % i for i in range(max_classes)]}
for scr_scen in scratch_scenarios_list:
    nc_first_task = scr_scen[0]
    nc_incr_tasks = scr_scen[1]
    scratch_df_path = glob(scratch_df_path_dict[scr_scen])
    tmp_df = load_scratch_data(scratch_df_path, base=nc_first_task, incr=nc_incr_tasks)
    tmp_df = tmp_df[['Seed', 'Approach', 'Base Apps', 'Increment', 'Episode', 'F1 Score', 'Class order', 'Last App']]
    num_tasks = tmp_df['Episode'].unique().shape[0]
    tmp_df_groups = tmp_df.groupby(['Seed', 'Approach', 'Base Apps', 'Increment', 'Last App'])
    for gr in tmp_df_groups.groups:
        per_ep_df = tmp_df_groups.get_group(gr).reset_index(drop=True)
        #print(per_ep_df)
        for i in range(num_tasks):
            exp_info_dict['Seed'].append(per_ep_df['Seed'].iloc[i])
            exp_info_dict['Approach'].append('scratch')
            exp_info_dict['Task'].append(per_ep_df['Episode'].iloc[i])
            exp_info_dict['Nc first task'].append(per_ep_df['Base Apps'].iloc[i])
            exp_info_dict['Nc incr tasks'].append(per_ep_df['Increment'].iloc[i])
            exp_info_dict['Last App'].append(str(per_ep_df['Last App'].iloc[i]))
            exp_info_dict['Metric'].append('F1 Score')
            class_order = per_ep_df['Class order'].iloc[i]
            #print(class_order)
            per_class_met_arrival_order_dict['Class order'].append(class_order)
            for cl in range(max_classes):
                class_pos = class_order.index(cl)
                #print(class_pos)
                #input('check class pos')
                per_class_met_arrival_order_dict['Class order[%s]' % class_pos].append(per_ep_df['F1 Score'].iloc[i][class_pos] if cl in class_order[:nc_first_task+(i*nc_incr_tasks)] else np.nan)
                per_class_met_dict['Class %s' % cl].append(per_ep_df['F1 Score'].iloc[i][class_pos] if cl in class_order[:nc_first_task+(i*nc_incr_tasks)] else np.nan)
            #print(per_class_met_arrival_order_dict)
            #print(per_class_met_dict)
            #input('check')
exp_info_per_cl_met_ord_arrival_dict = {**exp_info_dict, **per_class_met_arrival_order_dict}
exp_info_per_cl_met_dict = {**exp_info_dict, **per_class_met_dict}

scratch_met_ord_arrival_df = pd.DataFrame(exp_info_per_cl_met_ord_arrival_dict)
scratch_met_df = pd.DataFrame(exp_info_per_cl_met_dict)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 116.04it/s]


In [15]:
scratch_met_ord_arrival_df

,Seed,Approach,Task,Nc first task,Nc incr tasks,Last App,Metric,Class order[0],Class order[1],Class order[2],...,Class order[31],Class order[32],Class order[33],Class order[34],Class order[35],Class order[36],Class order[37],Class order[38],Class order[39],Class order
0,0,scratch,0,39,1,0,F1 Score,0.730906,0.787444,0.587379,...,0.893617,0.822857,0.881621,0.788882,0.822157,0.824813,0.881949,0.722353,NaN,"[10, 26, 12, 23, 21, 7, 27, 17, 35, 1, 36, 30,..."
1,0,scratch,0,39,1,1,F1 Score,0.734633,0.715564,0.885410,...,0.710128,0.584577,0.607634,0.980613,0.834320,0.854342,0.686003,0.716294,NaN,"[25, 24, 4, 5, 32, 12, 17, 38, 8, 7, 6, 16, 31..."
2,0,scratch,0,39,1,2,F1 Score,0.726392,0.837349,0.590085,...,0.768627,0.898072,0.794393,0.981318,0.781792,0.786127,0.843750,0.591304,NaN,"[13, 29, 30, 22, 25, 31, 39, 9, 36, 24, 3, 32,..."
3,0,scratch,0,39,1,3,F1 Score,0.879473,0.829268,0.576087,...,0.980613,0.733628,0.774660,0.881750,0.701389,0.845390,0.855556,0.584158,NaN,"[23, 2, 12, 10, 31, 24, 30, 0, 1, 17, 32, 34, ..."
4,0,scratch,0,39,1,4,F1 Score,0.615385,0.861582,0.681013,...,0.692308,0.731396,0.739040,0.729459,0.847761,0.602740,0.555249,0.893048,NaN,"[15, 16, 37, 0, 32, 2, 26, 18, 14, 8, 34, 13, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,38,scratch,1,20,20,-1,F1 Score,0.980170,0.773087,0.660465,...,0.888889,0.724919,0.887405,0.607171,0.827216,0.871300,0.756055,0.780952,0.605505,"[35, 0, 22, 17, 6, 9, 5, 34, 29, 37, 12, 36, 3..."
156,39,scratch,0,20,20,-1,F1 Score,0.764552,0.902394,0.935294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[3, 2, 21, 23, 10, 16, 18, 25, 33, 39, 1, 15, ..."
157,39,scratch,1,20,20,-1,F1 Score,0.701879,0.844920,0.881935,...,0.716883,0.744977,0.790000,0.882793,0.974760,0.901449,0.484029,0.810026,0.502484,"[3, 2, 21, 23, 10, 16, 18, 25, 33, 39, 1, 15, ..."
158,40,scratch,0,20,20,-1,F1 Score,0.829114,0.840140,0.798561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[25, 22, 24, 29, 2, 16, 4, 5, 35, 36, 13, 34, ..."


In [16]:
#scratch_met_ord_arrival_df.groupby(['Nc first task', 'Nc incr tasks']).get_group((39,1))['Seed']

In [17]:
#print(scratch_met_ord_arrival_df['Nc first task'].unique())
#print(scratch_met_ord_arrival_df['Nc incr tasks'].unique())

In [18]:
#scratch_met_ord_arrival_df[scratch_met_ord_arrival_df['Nc first task'] == 40]

In [19]:
met_ord_arrival_df = pd.concat([per_cl_met_ord_arrival_df, scratch_met_ord_arrival_df])
met_df = pd.concat([per_cl_met_dict_df, scratch_met_df])
met_ord_arrival_df.to_parquet(os.path.join(res_root_path, 'mirage_generic_per_class_arrival_order_metrics.parquet'))
met_df.to_parquet(os.path.join(res_root_path, 'mirage_generic_per_class_metrics.parquet'))